In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/coco_preprocessed.zip'  # Adjust if needed
extract_path = '/content/coco_preprocessed'

# Unzip the dataset
from zipfile import ZipFile

with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print('Dataset extracted successfully.')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted successfully.


In [ ]:
!pip install transformers pillow torch pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 882.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load processor and model
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = AutoModelForImageTextToText.from_pretrained("Salesforce/blip-image-captioning-base")

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [ ]:
# Path to a sample image
sample_image_path ='/content/coco_preprocessed/test/images/000000033213.jpg'  # Change this to a real image in your dataset

# Load and process the image
image = Image.open(sample_image_path).convert('RGB')
inputs = processor(images=image, return_tensors="pt").to(device)

# Generate the caption
outputs = model.generate(**inputs, max_new_tokens=50)
caption = processor.batch_decode(outputs, skip_special_tokens=True)[0]

print(f"Generated Caption: {caption}")

Generated Caption: a street sign in the middle of a city


In [ ]:
import json

def inspect_converted_file(path, num_entries=5):
    try:
        with open(path, 'r') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Failed to load JSON: {e}")
        return

    if not isinstance(data, list):
        print("JSON root is not a list.")
        return

    print(f"Loaded {len(data)} entries from {path}")
    print(f"First {num_entries} entries:\n")

    for i, item in enumerate(data[:num_entries]):
        if not isinstance(item, dict):
            print(f"Entry {i} is not a dictionary.")
            continue
        image = item.get("image")
        caption = item.get("caption")
        print(f"{i+1}. image = {image}\n   caption = {caption}\n")

inspect_converted_file("/content/coco_preprocessed/train/captions.json")
# inspect_converted_file("/content/dataset/val_converted.json")
# inspect_converted_file("/content/dataset/test_converted.json")

JSON root is not a list.


In [ ]:
# Try to auto-repair missing closing brace or extra comma
with open("/content/coco_preprocessed/test/captions.json", "r") as f:
    content = f.read()

# Try to auto-fix common issues
if content.strip()[-1] == ',':
    print("Detected trailing comma, attempting to fix...")
    content = content.rstrip(', \n') + '\n}'

if not content.strip().endswith('}'):
    print("Missing closing brace, attempting to fix...")
    content += '}'

try:
    data = json.loads(content)
    print(f"Successfully loaded JSON with {len(data)} entries.")
except json.JSONDecodeError as e:
    print(f"Failed to fix: {e}")


Successfully loaded JSON with 11827 entries.


In [ ]:
with open('/content/coco_preprocessed/test/captions.json', 'r') as f:
    content = f.read()

print("Start preview:")
print(content[:500])  # First 500 characters
print("\nEnd preview:")
print(content[-500:])  # Last 500 characters

Start preview:
{"000000033213.jpg": ["A neon green street sign on a sidewalk.", "A yellow sign that is on the sidewalk.", "A pedestrian crossing shows an arrow pointing low down.", "A yellow pedestrian sign indicating a low step down.", "A pole with two different street signs on it. "], "000000426972.jpg": ["A sparse room with a bed sitting in the corner.", "A bed in a room with wooden floor and not much else in the room.", "A bedroom with green walls and a bed with two pillows.", "A bedroom scene with focus o

End preview:
n in a red wet suit top carries his red surfboard on his head. ", "A guy carries his surfboard on his head.", "Man in red carrying a red surf board over his head.", "A man in a wet suit holding a surfboard over his head."], "000000457825.jpg": ["A home with white walls and rooms inside.", "A hallway with a couple of door ways.", "A hallway leading to an open door of a room filled with items.", "A hallway with three doors and items in the rooms.", "A short hallway wi

In [ ]:
import json

json_path = '/content/coco_preprocessed/test/captions.json'
merged_dict = {}

with open(json_path, 'r') as f:
    for line in f:
        try:
            entry = json.loads(line.strip())
            merged_dict.update(entry)
        except Exception as e:
            print(f"Error reading line: {e}")

print(f"Successfully loaded {len(merged_dict)} entries.")

# Save the fixed JSON
output_path = '/content/coco_preprocessed/test/captions_fixed.json'
with open(output_path, 'w') as f:
    json.dump(merged_dict, f, indent=2)

print(f"Fixed JSON saved to {output_path}")

Successfully loaded 11827 entries.
Fixed JSON saved to /content/coco_preprocessed/test/captions_fixed.json


In [ ]:
import json

json_path = '/content/coco_preprocessed/test/captions.json'

# Load the entire JSON object
with open(json_path, 'r') as f:
    try:
        merged_dict = json.load(f)
        print(f"Successfully loaded {len(merged_dict)} entries.")
    except Exception as e:
        print(f"Error reading JSON file: {e}")
        merged_dict = {}

# Save the JSON as is (optional, if you wanted to "fix" formatting)
output_path = '/content/coco_preprocessed/test/captions_fixed.json'
with open(output_path, 'w') as f:
    json.dump(merged_dict, f, indent=2)

print(f"Fixed JSON saved to {output_path}")

Successfully loaded 11827 entries.
Fixed JSON saved to /content/coco_preprocessed/test/captions_fixed.json


In [ ]:
pip install evaluate

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
import json
from PIL import Image
from evaluate import load

# Load BERTScore metric
bertscore = load('bertscore')

# Load the captions JSON
json_path = '/content/coco_preprocessed/test/captions_fixed.json'
with open(json_path, 'r') as f:
    captions_dict = json.load(f)

# Path to the sample image
sample_image_path = '/content/coco_preprocessed/test/images/000000033213.jpg'

# Extract the image filename
image_filename = sample_image_path.split('/')[-1]

# Get the ground truth captions
ground_truth_captions = captions_dict.get(image_filename, ["No captions found for this image."])

# Load and process the image
image = Image.open(sample_image_path).convert('RGB')
inputs = processor(images=image, return_tensors="pt").to(device)

# Generate the caption
outputs = model.generate(**inputs, max_new_tokens=50)
generated_caption = processor.batch_decode(outputs, skip_special_tokens=True)[0]

# Prepare references for BERTScore: a list of strings (best if you combine multiple ground truths into one string per reference)
references = [' '.join(ground_truth_captions)]  # join all reference captions into one string

# Compute BERTScore
results = bertscore.compute(predictions=[generated_caption], references=references, lang='en')

# Print both captions and BERTScore
print(f"\nGenerated Caption: {generated_caption}")
print("\nGround Truth Captions:")
for idx, caption in enumerate(ground_truth_captions, 1):
    print(f"{idx}. {caption}")

print(f"\nBERTScore (F1): {results['f1'][0]:.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Generated Caption: a street sign in the middle of a city

Ground Truth Captions:
1. A neon green street sign on a sidewalk.
2. A yellow sign that is on the sidewalk.
3. A pedestrian crossing shows an arrow pointing low down.
4. A yellow pedestrian sign indicating a low step down.
5. A pole with two different street signs on it. 

BERTScore (F1): 0.8871
